In [ ]:
#TfidfVectorizer

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
text = ['Hello Sagar Honnabindagi here','Welcome to the Fake News Detection' ]

In [2]:
vect = TfidfVectorizer()

In [3]:
vect.fit(text)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [4]:
## TF will count the frequency of word in each document. and IDF 
print(vect.idf_)

[1.40546511 1.40546511 1.40546511 1.40546511 1.40546511 1.40546511
 1.40546511 1.40546511 1.40546511 1.40546511]


In [5]:
print(vect.vocabulary_)

{'hello': 2, 'sagar': 6, 'honnabindagi': 4, 'here': 3, 'welcome': 9, 'to': 8, 'the': 7, 'fake': 1, 'news': 5, 'detection': 0}


In [6]:
example = text[0]
example

'Hello Sagar Honnabindagi here'

In [7]:
example = vect.transform([example])
print(example.toarray())

[[0.  0.  0.5 0.5 0.5 0.  0.5 0.  0.  0. ]]


In [8]:
import pandas as pd

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
dataframe = pd.read_csv('/content/drive/MyDrive/FakeNewsDetection/implementation2/news.csv')
dataframe.head(10)

Unnamed: 0  ... label
0        8476  ...  FAKE
1       10294  ...  FAKE
2        3608  ...  REAL
3       10142  ...  FAKE
4         875  ...  REAL
5        6903  ...  FAKE
6        7341  ...  FAKE
7          95  ...  REAL
8        4869  ...  REAL
9        2909  ...  REAL

[10 rows x 4 columns]

In [12]:
x = dataframe['text']
y = dataframe['label']

In [13]:
x

0       Daniel Greenfield, a Shillman Journalism Fello...
1       Google Pinterest Digg Linkedin Reddit Stumbleu...
2       U.S. Secretary of State John F. Kerry said Mon...
3       — Kaydee King (@KaydeeKing) November 9, 2016 T...
4       It's primary day in New York and front-runners...
                              ...                        
6330    The State Department told the Republican Natio...
6331    The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
6332     Anti-Trump Protesters Are Tools of the Oligar...
6333    ADDIS ABABA, Ethiopia —President Obama convene...
6334    Jeb Bush Is Suddenly Attacking Trump. Here's W...
Name: text, Length: 6335, dtype: object

In [14]:
y

0       FAKE
1       FAKE
2       REAL
3       FAKE
4       REAL
        ... 
6330    REAL
6331    FAKE
6332    FAKE
6333    REAL
6334    REAL
Name: label, Length: 6335, dtype: object

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [16]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)
y_train

2402    REAL
1922    REAL
3475    FAKE
6197    REAL
4748    FAKE
        ... 
4931    REAL
3264    REAL
1653    FAKE
2607    FAKE
2732    REAL
Name: label, Length: 5068, dtype: object

In [17]:
y_train

2402    REAL
1922    REAL
3475    FAKE
6197    REAL
4748    FAKE
        ... 
4931    REAL
3264    REAL
1653    FAKE
2607    FAKE
2732    REAL
Name: label, Length: 5068, dtype: object

In [18]:
tfvect = TfidfVectorizer(stop_words='english',max_df=0.7)
tfid_x_train = tfvect.fit_transform(x_train)
tfid_x_test = tfvect.transform(x_test)

In [19]:
classifier = PassiveAggressiveClassifier(max_iter=50)
classifier.fit(tfid_x_train,y_train)

PassiveAggressiveClassifier(C=1.0, average=False, class_weight=None,
                            early_stopping=False, fit_intercept=True,
                            loss='hinge', max_iter=50, n_iter_no_change=5,
                            n_jobs=None, random_state=None, shuffle=True,
                            tol=0.001, validation_fraction=0.1, verbose=0,
                            warm_start=False)

In [20]:
y_pred = classifier.predict(tfid_x_test)
score = accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 93.29%


In [21]:
cf = confusion_matrix(y_test,y_pred, labels=['FAKE','REAL'])
print(cf)

[[567  48]
 [ 37 615]]


In [22]:
def fake_news_det(news):
    input_data = [news]
    vectorized_input_data = tfvect.transform(input_data)
    prediction = classifier.predict(vectorized_input_data)
    print(prediction)

In [23]:
fake_news_det('U.S. Secretary of State John F. Kerry said Monday that he will stop in Paris later this week, amid criticism that no top American officials attended Sundayâ€™s unity march against terrorism.')

['REAL']


In [24]:
fake_news_det('Iranian negotiators reportedly have made a last-ditch push for more concessions from the U.S. and five other world powers as talks on the fate of Irans nuclear program come down to the final days before a crucial deadline.')

['REAL']


In [25]:
fake_news_det('Share This Baylee Luciani (left), Screenshot of what Baylee caught on FaceTime (right) The closest Baylee Luciani could get to her boyfriend, whoâ€™s attending college in Austin, was through video online chat. The couple had regular â€œdatesâ€ this way to bridge the 200-mile distance between them. However, the endearing arrangement quickly came to an end after his FaceTime was left on and caught something that left his girlfriend horrified. Baylee had been discussing regular things with her boyfriend, Yale Gerstein, who was on the other side of the screen on an otherwise average evening. This video chat was not unlike all the others she had with Yale from his apartment near Austin Community College until the 19-year-old girlfriend heard some scratching sounds after FaceTime had been left on. According to KRON , Baylee was mid-conversation with Yale when scratches at the door caught both of their attention and he got up from his bed, where the computer was, to see who was at his door. He barely turned the handle to open in when masked men entered the room and beat Yaleâ€™s face in and slammed him down on his bed while shoving a pistol in his cheek. The intruders didnâ€™t seem to know or care that FaceTime was still on and Bayleeâ€™s face, seen in the corner, was watching everything, terrified that she was about to see her boyfriend murdered in front of her, as she watched him fight for his life. Admitting that she first thought it was a joke, seconds later, she came to the horrid realization that he was being robbed and called her dad, who was at home with her in Dallas, into the room. â€œI was scared, because they were saying Iâ€™m going to blow your head off, Iâ€™m going to kill you,â€ Baylee explained along with the chilling feeling she got when the intruder finally realized the video chat was running and looked right at her in the camera. â€œIâ€™m like wowâ€¦ seriously watching an armed robbery happen to somebody that I care about,â€ she added. Screengrabs of intruder forcing Yale down on his bed while Baylee and her father watch on FaceTime in horror With a clear view of at least one intruderâ€™s face, Baylee began taking screenshots of the suspect in the act as she and her dad called the police to report what was going on. She got the pictures right in time since, seconds later, the intruder decided to disconnect the computer as he and the suspects took off with thousands of dollars worth of Yaleâ€™s music equipment. Although the boyfriendâ€™s life was spared in the traumatizing ordeal for the two of them, he said that the thieves took something from him that canâ€™t be replaced. â€œI had just finished my first album as a solo artist,â€ Yale said. â€œThatâ€™s all lost,â€ since they took the recordings on the equipment, which means nothing to the thieves and everything to the victim. Itâ€™s not often that you hear of FaceTime solving crimes or potentially saving lives, which is what happened in this case. Although it was difficult to watch, Baylee, being there through technology, was an instrumental part in protecting Yale, who hopefully learned that he better take advantage of Texasâ€™ great gun laws and arm himself with more than just a computer.')

['FAKE']


In [26]:
import pickle
pickle.dump(classifier,open('model.pkl', 'wb'))

In [27]:
# load the model from disk
loaded_model = pickle.load(open('model.pkl', 'rb'))

In [28]:
def fake_news_det1(news):
    input_data = [news]
    vectorized_input_data = tfvect.transform(input_data)
    prediction = loaded_model.predict(vectorized_input_data)
    print(prediction)

In [29]:
fake_news_det1("""Go to Article 
President Barack Obama has been campaigning hard for the woman who is supposedly going to extend his legacy four more years. The only problem with stumping for Hillary Clinton, however, is sheâ€™s not exactly a candidate easy to get too enthused about.  """)

['FAKE']


In [30]:
fake_news_det1("""U.S. Secretary of State John F. Kerry said Monday that he will stop in Paris later this week, amid criticism that no top American officials attended Sundayâ€™s unity march against terrorism.""")

['REAL']
